# Exploring the OOI ERDDAP server

In [1]:
from erddapy import ERDDAP


server = "https://erddap-uncabled.oceanobservatories.org/uncabled/erddap"
e = ERDDAP(server=server)

# What services are available in the **erddap-uncabled.oceanobservatories** server?

In [2]:
import pandas as pd


df = pd.read_csv(
    e.get_search_url(response='csv', search_for='all')
)

In [3]:
print(
    f'We have {len(set(df["tabledap"].dropna()))} '
    f'tabledap, {len(set(df["griddap"].dropna()))} '
    f'griddap, and {len(set(df["wms"].dropna()))} wms endpoints.'
)

We have 1000 tabledap, 0 griddap, and 0 wms endpoints.


# Let's query all the salinity data available on this server

In [4]:
url = e.get_categorize_url(
    categorize_by="standard_name",
    value="sea_water_practical_salinity",
    response="csv"
)

df = pd.read_csv(url)
dataset_ids = df.loc[~df['tabledap'].isnull(), 'Dataset ID'].tolist()

dataset_ids_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} Datasets:\n{dataset_ids_list}")

Found 343 Datasets:
CE05MOAS-GL247-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap
CE05MOAS-GL311-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0004-tabledap
CE05MOAS-GL311-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap
CE05MOAS-GL311-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0007-tabledap
CE05MOAS-GL312-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap
CE05MOAS-GL319-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap
CE05MOAS-GL319-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0007-tabledap
CE05MOAS-GL319-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0008-tabledap
CE05MOAS-GL320-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0004-tabledap
CE05MOAS-GL327-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0005-tabledap
CE05MOAS-GL327-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006

# We don't really want all that, let's get all the salinity data within a lon/lat/time extent

In [5]:
kw = {
    "min_time": "2018-08-05T00:00:00Z",
    "min_time": "2019-06-12T00:00:00Z",
    "min_lon": -133.75,
    "max_lon": -123.29,
    "min_lat": 41.78,
    "max_lat": 52.24,
    "standard_name": "sea_water_practical_salinity",
}

In [6]:
search_url = e.get_search_url(response="csv", **kw)
search = pd.read_csv(search_url)
dataset_ids = search["Dataset ID"].values

dataset_ids_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} Datasets:\n{dataset_ids_list}")

Found 16 Datasets:
CE05MOAS-GL384-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap
CE01ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE01ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE01ISSM-SBD17-06-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE04OSSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0008-tabledap
CE04OSSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0008-tabledap
CE02SHSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0009-tabledap
CE02SHSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0009-tabledap
CE06ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap
CE06ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap
CE06ISSM-SBD17-06-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabled

# Based on the ids names we have a mix of glider and fixed stations, let's find out!

In [7]:
for dataset_id in dataset_ids:
    url = e.get_info_url(dataset_id=dataset_id, response="csv")
    df = pd.read_csv(url)
    val = df.loc[df["Attribute Name"] == "cdm_data_type"]["Value"].squeeze()
    print(f"{dataset_id}: {val}")

CE05MOAS-GL384-05-CTDGVM000-ctdgv_m_glider_instrument-telemetered-deployment0006-tabledap: Trajectory
CE01ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap: Point
CE01ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap: Point
CE01ISSM-SBD17-06-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap: Point
CE04OSSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0008-tabledap: Point
CE04OSSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0008-tabledap: Point
CE02SHSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0009-tabledap: Point
CE02SHSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0009-tabledap: Point
CE06ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap: Point
CE06ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap: Point
CE06ISSM-SBD17-06-CTDBPC000-ctd

In [8]:
kw = {
    "min_time": "2018-08-05T00:00:00Z",
    "min_time": "2019-06-12T00:00:00Z",
    "min_lon": -133.75,
    "max_lon": -123.29,
    "min_lat": 41.78,
    "max_lat": 52.24,
    "standard_name": "sea_water_practical_salinity",
    "cdm_data_type": "point",
}

search_url = e.get_search_url(response="csv", **kw)
search = pd.read_csv(search_url)
dataset_ids = search["Dataset ID"].values

dataset_ids_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} Datasets:\n{dataset_ids_list}")

Found 15 Datasets:
CE01ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE01ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE01ISSM-SBD17-06-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0011-tabledap
CE04OSSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0008-tabledap
CE04OSSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0008-tabledap
CE02SHSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0009-tabledap
CE02SHSM-SBD11-06-METBKA000-metbk_a_dcl_instrument-telemetered-deployment0009-tabledap
CE06ISSM-RID16-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap
CE06ISSM-MFD37-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap
CE06ISSM-SBD17-06-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0010-tabledap
CE09OSSM-RID27-03-CTDBPC000-ctdbp_cdef_dcl_instrument-telemetered-deployment0009-tabled

In [9]:
set(df["Variable Name"])

{'NC_GLOBAL',
 'air_temperature',
 'barometric_pressure',
 'ct_depth',
 'dcl_controller_timestamp',
 'deployment',
 'driver_timestamp',
 'eastward_wind_velocity',
 'id',
 'ingestion_timestamp',
 'internal_timestamp',
 'latitude',
 'longitude',
 'longwave_irradiance',
 'met_barpres',
 'met_current_direction',
 'met_current_speed',
 'met_heatflx_minute',
 'met_latnflx_minute',
 'met_netlirr_minute',
 'met_netsirr',
 'met_relwind_direction',
 'met_relwind_speed',
 'met_salsurf',
 'met_sensflx_minute',
 'met_spechum',
 'met_windavg_mag_corr_east',
 'met_windavg_mag_corr_north',
 'northward_wind_velocity',
 'port_timestamp',
 'precipitation',
 'preferred_timestamp',
 'provenance',
 'quality_flag',
 'relative_humidity',
 'sea_surface_conductivity',
 'sea_surface_temperature',
 'shortwave_irradiance',
 'time'}

Exercise: get the glider